In [2]:
import datetime
import numpy as np
import pandas_datareader as pdr
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
from utils.leadlag import leadlag #USE THIS LINE FOR JUPYTER NOTEBOOK
#from leadlag import leadlag #USE THIS LINE FOR GOOGLE COLAB

import import_ipynb
from cvae import CVAE


class MarketGenerator:
    def __init__(self, ticker, start=datetime.date(1970, 1, 1),
                 end=datetime.date(2021, 5, 14), freq="M"):

        self.ticker = ticker
        self.start = start
        self.end = end
        self.freq = freq

        self._load_data()

        self._build_dataset()
        self.generator = CVAE(n_latent=2, alpha=0.003)


    def _load_data(self):
        try:
            self.data = pdr.get_data_yahoo(self.ticker, self.start, self.end)["Close"]
        except:
            raise RuntimeError(f"Could not download data for {self.ticker} from {self.start} to {self.end}.")
        
        #self.data = spx_df[self.start <= spx_df['Date'] <= self.end][['Date', 'Close']].set_index("Date").squeeze()
        
        
        self.windows = []
        for _, window in self.data.resample(self.freq):
            values = window.values 
            path = leadlag(values)

            self.windows.append(path)

    def _build_dataset(self):
        # Calculates log-returns of path (no signatures)
        temp_path = self.windows[0]

        self.orig_logsig = np.array([np.diff(np.log(path[::2, 1])) for path in self.windows])
        self.orig_logsig = np.array([p for p in self.orig_logsig if len(p) >= 4])
        steps = min(map(len, self.orig_logsig))
        self.orig_logsig = np.array([val[:steps] for val in self.orig_logsig])

        self.scaler = MinMaxScaler(feature_range=(0.00001, 0.99999))
        logsig = self.scaler.fit_transform(self.orig_logsig)

        self.logsigs = logsig[1:]
        self.conditions = logsig[:-1]

    def train(self, n_epochs=10000):
        self.generator.train(self.logsigs, self.conditions, n_epochs=n_epochs)

    def generate(self, logsig, n_samples=None, normalised=False):
        generated = self.generator.generate(logsig, n_samples=n_samples)

        if normalised:
            return generated

        if n_samples is None:
            return self.scaler.inverse_transform(generated.reshape(1, -1))[0]

        return self.scaler.inverse_transform(generated)
    
    

importing Jupyter notebook from cvae.ipynb


c:\users\dorot\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\dorot\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\dorot\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\dorot\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi